In [26]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics
from calitp_data_analysis import calitp_color_palette

# from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import segment_speed_utils

import altair as alt

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [139]:
## parameters cell
itp_id = 182

In [140]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt

In [141]:
# speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet')

In [142]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)

In [143]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [144]:
speedmap_segs = gpd.read_parquet(path) #  aggregated

In [145]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [146]:
#  TODO to utils
def time_period_and_arrowize(gdf, time_period):
    
    gdf = gdf >> filter(_.time_of_day == time_period)
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1, frequency_col='trips_hr_sch')

    gdf = gdf >> arrange(_.trips_hr_sch)

    return gdf

In [151]:
time_of_day = 'AM Peak'

In [155]:
period_test = time_period_and_arrowize(speedmap_segs, time_of_day)

In [156]:
cmap = shared_utils.rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.ACCESS_SPEEDMAP_LEGEND_URL

In [157]:
period_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,direction_id,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch,route_short_name
10501,0666caf3ec1ecc96b74f4477ee4bc939,fedc605c108af558e4e545806741ea0f,510358_JUNE24,51-13183,0.0,283__8824,Avalon / Imperial__Avalon / 111th,283-8824-1,AM Peak,12.34,...,12.34,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"POLYGON ((160449.331 -452365.708, 160434.047 -...",1,0.333333,51
10502,0666caf3ec1ecc96b74f4477ee4bc939,fedc605c108af558e4e545806741ea0f,510358_JUNE24,51-13183,0.0,8824__8823,Avalon / 111th__Avalon / 110th,8824-8823-1,AM Peak,14.66,...,14.66,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"POLYGON ((160443.819 -452040.084, 160428.472 -...",1,0.333333,51


## export map

In [158]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

In [159]:
#  TODO clone render method from RtFilterMapper, to utils
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/182_am_peak_new.geojson.gz


## docs!

In [167]:
domain = cmap.index

In [168]:
range_ = [cmap.rgb_hex_str(i) for i in cmap.index]

In [169]:
df = speedmap_segs[['time_of_day', 'p50_mph', 'p20_mph', 'p80_mph']]
df = df >> group_by(_.time_of_day) >> summarize(p50_mph = _.p50_mph.quantile(.5),
                                               p20_mph = _.p20_mph.quantile(.5),
                                               p80_mph = _.p80_mph.quantile(.5),)
df['p50 - p20'] = -(df['p50_mph'] - df['p20_mph'])
df['p80 - p50'] = df['p80_mph'] - df['p50_mph']

In [178]:
error_bars = alt.Chart(df).mark_errorbar(thickness=5, color='gray', opacity=.6).encode(
    y = alt.Y("p50_mph:Q", title='Segment Speed: 20, 50, 80%ile'),
    yError=("p50 - p20:Q"),
    yError2=("p80 - p50:Q"),
    x = alt.X("time_of_day:N", sort=['Early AM', 'AM Peak', 'Midday', 'PM Peak', 'Evening', 'Owl']),
    tooltip=[alt.Tooltip('p20_mph:Q', title="p20 mph"), alt.Tooltip('p50_mph:Q', title="p50 mph"),
            alt.Tooltip('p80_mph:Q', title="p80 mph")]
).properties(width=400)

points = alt.Chart(df).mark_point(filled=True, size = 300, opacity = 1).encode(
    alt.Y("p50_mph:Q"),
    alt.X("time_of_day:N", sort=['Early AM', 'AM Peak', 'Midday', 'PM Peak', 'Evening', 'Owl'],
         title='Time of Day'),
    color=alt.Color('p50_mph', title='Median Segment Speed (mph)').scale(domain=domain, range = range_),
    tooltip=[alt.Tooltip('p50_mph:Q', title="p50 mph")],
)

chart = error_bars + points
chart = chart.configure(axis = alt.AxisConfig(labelFontSize=14, titleFontSize=18),
                       legend = alt.LegendConfig(titleFontSize=14, labelFontSize=14, titleLimit=250,
                                                 titleOrient='left', labelOffset=100))

In [180]:
chart

alt.LayerChart(...)

In [138]:
chart

alt.LayerChart(...)